In [1]:
from fasttext_comparison_function import eval_dataset

In [2]:
import fasttext
import os
import sys
import os.path
import numpy as np
from numpy import array
from sklearn.model_selection import KFold
import json
import pandas as pd
from sklearn.model_selection import train_test_split
import neattext.functions as nfx
import re

In [4]:
import pandas as pd
# custom data set
data_in_path = r"..\dataset\CLEAN_balanced_data.csv"
data_out_path = r"..\results\output.txt"
df = pd.read_csv(data_in_path)
df = df.dropna()
# df['data'] = "__label__" + df['labels'] + " " + df['descriptions']
# df['data'] = df['labels'] + " " + df['descriptions']
df.head()


,labels,descriptions,repo_name
0,__label__enhancement,"""script path builtin variable (a la `__file__`...",other
1,__label__enhancement,"""give lobby creator extra powers hey",other
2,__label__enhancement,dmtmm congratulations! guys great works past h...,other
3,__label__enhancement,"""router - avoid regex matching possible given ...",other
4,__label__enhancement,"""improve command history currently",other


In [5]:
data_path = r"..\dataset\ft_bal.txt"
# df['data'] = "__label__" + df['labels'] + " " + df['descriptions']
df['data'] = df['labels'] + " " + df['descriptions']
df_file = df[['data']]
df_file.to_csv(data_path,index=False,header=False,sep="\t",encoding='utf-8-sig')
df_file.shape

(29625, 1)

# testing initial model

In [8]:

# path_train = os.path.dirname(__file__) + './tmp/tmp_train.txt'
# path_test = os.path.dirname(__file__) + './tmp/tmp_test.txt'

path_test = r"..\temp\temp_eval.txt"
path_train = r"..\temp\temp_train.txt"
data_out_path = r"..\results\output_testing_output.txt"
data_set = data_path
f_out = data_out_path
# try:
print("Converting dataset to array")
# f = open(data_set, 'r+', encoding="UTF-8")

data = array(df_file['data'])
# f.close()

# array for details
fold_outputs = []

# 10 fold loop
kfold = KFold(10, shuffle=True, random_state=1)
fold = 1
for train, test in kfold.split(data):
    print("New tenfold iteration:", str(fold), "-----------------------------------------")
    print("Creating train file")
    tmp_train = data[train]
    pd.DataFrame(tmp_train).to_csv(path_train,index=False,header=False,encoding='utf-8-sig')

    print("Creating test file")
    tmp_test = data[test]
    pd.DataFrame(tmp_test).to_csv(path_test,index=False,header=False,encoding='utf-8-sig')



    print("start training...")
    # use this model if you want to enable auto-tuning. Sadly we did not get it to work during our project
    # model = fasttext.train_supervised(input=path_train, autotuneValidationFile=path_test, autotuneDuration=600)
    # model = fasttext.train_supervised(input=path_train)  # normal fasttext classification without autotuning
    model = fasttext.train_supervised(input=path_train, lr=1.0, epoch=25, wordNgrams=5)
    print("start testing...")
    res = model.test(path_test)
    print(res)
    # get benchmarks
    precision = res[1]
    recall = res[2]
    f1 = 2 * ((precision * recall) / (precision + recall))
    result = {
        '10-Fold iteration:': fold,
        'F1': f1,
        'Recall': recall,
        'Precision': precision
        # 'acuracy'
    }
    # log
    print("Fold over, here are results: ")
    print(json.dumps(result, indent=4))
    fold_outputs.append(result)
    fold += 1
print("Done with 10 fold validation")
# calculate over-all results
mean_recall = 0
mean_precision = 0
mean_f1 = 0
for f in fold_outputs:
    mean_f1 += (f['F1'] / 10)
    mean_recall += (f['Recall'] / 10)
    mean_precision += (f['Precision'] / 10)
    # compile results as json
output = {
    'Results': {
        'F1': mean_f1,
        'Recall': mean_recall,
        'Precision': mean_precision
    },
    'Details': fold_outputs
}
dump = json.dumps(output, indent=4)
print(dump)
# write to output
print("Writing output to file")
o = open(f_out, 'w', encoding="UTF-8")
o.write(dump)
o.close()
# except:
# print("An Error occurred")
# # in any case delete existing temporary files
# finally:
print("Deleting tmp files")
if os.path.exists(path_train):
    os.remove(path_train)
if os.path.exists(path_test):
    os.remove(path_test)
print("Exit.")



Converting dataset to array
New tenfold iteration: 1 -----------------------------------------
Creating train file
Creating test file
start training...
start testing...
(651, 0.6497695852534562, 0.6497695852534562)
Fold over, here are results: 
{
    "10-Fold iteration:": 1,
    "F1": 0.6497695852534562,
    "Recall": 0.6497695852534562,
    "Precision": 0.6497695852534562
}
New tenfold iteration: 2 -----------------------------------------
Creating train file
Creating test file
start training...
start testing...
(652, 0.661042944785276, 0.661042944785276)
Fold over, here are results: 
{
    "10-Fold iteration:": 2,
    "F1": 0.661042944785276,
    "Recall": 0.661042944785276,
    "Precision": 0.661042944785276
}
New tenfold iteration: 3 -----------------------------------------
Creating train file
Creating test file
start training...
start testing...
(654, 0.6529051987767585, 0.6529051987767585)
Fold over, here are results: 
{
    "10-Fold iteration:": 3,
    "F1": 0.6529051987767585,

Plan / work steps
need to split the repo level data into test/train datasets

need to train a model with a simple test/ train split from the balanced data
then use that model to predict on each of the repo datasets

then split each repo dataset into test train

include the repo data inside the training data
then repredict with it
see if it improves it at all. 



Need to organize the data into separate groups for each of the different repo
then run the eval function over each of them. 



In [188]:
data_in_path = r"..\dataset\CLEAN_balanced_data.csv"
data_out_path = r"..\results\output_General_Model.txt"
f_out = data_out_path
bal_df = pd.read_csv(data_in_path)
bal_df = bal_df.dropna()
bal_df.head()

,labels,descriptions,repo_name
0,__label__enhancement,"""script path builtin variable (a la `__file__`...",other
1,__label__enhancement,"""give lobby creator extra powers hey",other
2,__label__enhancement,dmtmm congratulations! guys great works past h...,other
3,__label__enhancement,"""router - avoid regex matching possible given ...",other
4,__label__enhancement,"""improve command history currently",other


In [187]:
# custom data set
data_in_path = r"..\dataset\CLEAN_UNbalanced_data.csv"
data_out_path = r"..\results\output.txt"
un_df = pd.read_csv(data_in_path)
un_df = un_df.dropna()
un_df['labels'] = "__label__"+ un_df['labels'] 
un_df.head()

,labels,descriptions
0,__enhancement__,subpasses msl hlsl sorry issues given followi...
1,__enhancement__,collect new discovered nodes collect new added...
2,__enhancement__,lock property : properties
3,__enhancement__,randomize: remove sliders
4,__enhancement__,feature request: local invoke inside action co...


In [183]:


labels_list = list(bal_df['labels'].unique())
labels_list

['__label__enhancement', '__label__bug', '__label__question']

In [178]:
# un_df.head()
this = list(predicted_labels)
this = [label[0] for label in predicted_labels]
cm = confusion_matrix(y_true=actual_labels,y_pred=this,labels=labels_list)


ValueError: At least one label specified must be in y_true

# Creating General Model

In [49]:
import pandas as pd
# custom data set
path_test = r"..\temp\temp_eval.txt"
path_train = r"..\temp\temp_train.txt"
data_in_path = r"..\dataset\CLEAN_balanced_data.csv"
data_out_path = r"..\results\output_General_Model.txt"
f_out = data_out_path
bal_df = pd.read_csv(data_in_path)
bal_df = bal_df.dropna()
bal_df.head()

bal_df['data'] = bal_df['labels'] + " " + bal_df['descriptions']
bal_df = bal_df[['data']]

print("Creating train file")
bal_df.to_csv(path_train,index=False,header=False,encoding='utf-8-sig')


# custom data set
data_in_path = r"..\dataset\CLEAN_UNbalanced_data.csv"
data_out_path = r"..\results\output.txt"
un_df = pd.read_csv(data_in_path)
un_df = un_df.dropna()
un_df['labels'] = "__label__"+ un_df['labels'] 

# print("Creating train file")
# un_df.to_csv(path_test,index=False,header=False,encoding='utf-8-sig')

model_general = fasttext.train_supervised(input=path_train, lr=1.0, epoch=25, wordNgrams=5)




Creating train file


In [50]:
un_df.head()

,labels,descriptions
0,__label__enhancement,subpasses msl hlsl sorry issues given followi...
1,__label__enhancement,collect new discovered nodes collect new added...
2,__label__enhancement,lock property : properties
3,__label__enhancement,randomize: remove sliders
4,__label__enhancement,feature request: local invoke inside action co...


In [61]:

un_df_test = un_df
un_df_test['data'] = un_df_test['labels'] + " " + un_df_test['descriptions']
un_df_test = un_df_test[['data']]
un_df_test.to_csv(path_test,index=False,header=False,encoding='utf-8-sig')
res = model_general.test(path_test)
print(res)
# get benchmarks
precision = res[1]
recall = res[2]
f1 = 2 * ((precision * recall) / (precision + recall))
result = {
    '10-Fold iteration:': fold,
    'F1': f1,
    'Recall': recall,
    'Precision': precision
    # 'acuracy'
}
result

(8565, 0.736835960303561, 0.736835960303561)


{'10-Fold iteration:': 11,
 'F1': 0.7368359603035611,
 'Recall': 0.736835960303561,
 'Precision': 0.736835960303561}

In [53]:
print("start testing general model...")

predicted_labels = []
actual_labels =list(un_df['labels'])
for y in array(un_df['descriptions']):
    predicted_labels.append(model_general.predict(y))
predicted_labels = [label[0][0] for label in predicted_labels]


start testing general model...


In [58]:
data_out_path = r"..\results\output_General_Model.txt"
f_out = data_out_path
cm = confusion_matrix(actual_labels,predicted_labels,labels_list)

TP = np.diag(cm)
FP = np.sum(cm, axis=0) - TP
FN = np.sum(cm, axis=1) - TP

num_classes = 3
TN = []
for i in range(num_classes):
    temp = np.delete(cm, i, 0)    # delete ith row
    temp = np.delete(temp, i, 1)  # delete ith column
    TN.append(sum(sum(temp)))

precision = TP/(TP+FP)
recall = TP/(TP+FN)
accuracy = (TP+TN) / (TP + FP + FN + TN)

f1 = 2 * ((precision * recall) / (precision + recall))
result_balanced = {
    'labels_list': labels_list,
    'F1': list(f1),
    'Recall': list(recall),
    'Precision': list(precision),
    'accuracy': list(accuracy)
    }

dump = json.dumps(result_balanced, indent=4)
print(dump)
# write to output
print("Writing output to file")
# f_out = data_out_path + f"\REPO_output.txt"
o = open(f_out, 'w', encoding="UTF-8")
o.write(dump)
o.close()
# except:
# print("An Error occurred")
# # in any case delete existing temporary files
# finally:
print("Deleting tmp files")
if os.path.exists(path_train):
    os.remove(path_train)
if os.path.exists(path_test):
    os.remove(path_test)
print("Exit.")

{
    "labels_list": [
        "__label__enhancement",
        "__label__bug",
        "__label__question"
    ],
    "F1": [
        0.7018999091519532,
        0.715258164964878,
        0.4122703522669813
    ],
    "Recall": [
        0.743763602879625,
        0.6569775176267127,
        0.49837000814995924
    ],
    "Precision": [
        0.6644977937327051,
        0.7848855689764781,
        0.3515377982178787
    ],
    "accuracy": [
        0.7435958415437929,
        0.7328259835564314,
        0.881531562139023
    ]
}
Writing output to file
Deleting tmp files
Exit.


e:\Anaconda_New\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass labels=['__label__enhancement', '__label__bug', '__label__question'] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


# Separating creating model and using it to predict repo issues

In [75]:
import pandas as pd
# custom data set
data_in_path = r"..\dataset\CLEAN_balanced_data.csv"
data_out_path = r"..\results\output.txt"
data_df = pd.read_csv(data_in_path)
data_df = data_df.dropna()
data_df.head()

,labels,descriptions,repo_name
0,__label__enhancement,"""script path builtin variable (a la `__file__`...",other
1,__label__enhancement,"""give lobby creator extra powers hey",other
2,__label__enhancement,dmtmm congratulations! guys great works past h...,other
3,__label__enhancement,"""router - avoid regex matching possible given ...",other
4,__label__enhancement,"""improve command history currently",other


In [76]:
# split dataset to train and dev samples
RANDOM_SEED = 12
x_train, x_test, y_train, y_test = train_test_split(data_df['descriptions'], data_df['labels'],
                                                 test_size=0.3,
                                                 random_state=RANDOM_SEED)
df_train = pd.concat([x_train, y_train], axis=1).reset_index(drop=True)
df_eval = pd.concat([x_test, y_test], axis=1).reset_index(drop=True)

In [77]:
# print(df_train.shape)
# print(df_eval.shape)
# print(data_df.shape)

In [78]:
df_train['data'] = df_train['labels'] + " " + df_train['descriptions']
df_train = df_train[['data']]

df_eval['data'] = df_eval['labels'] + " " + df_eval['descriptions']
df_eval = df_eval[['data']]

In [79]:
print("Creating train file")
df_train.to_csv(path_train,index=False,header=False,encoding='utf-8-sig')

print("Creating test file")
df_eval.to_csv(path_test,index=False,header=False,encoding='utf-8-sig')

Creating train file
Creating test file


In [80]:
model = fasttext.train_supervised(input=path_train, lr=1.0, epoch=25, wordNgrams=2)

In [81]:
# # # save train and eval dataframes to csv
# # get repo data 
df_all = pd.read_csv(r"..\dataset\CLEAN_multiple_projects_benchmark_data.csv")
df_all.head()

,Unnamed: 0,labels,descriptions,repo_name
0,4625,__label__enhancement,clarify sdl_locktexture() description # bug re...,https://api.github.com/repos/libsdl-org/SDL
1,4655,__label__enhancement,[patch] add stadia controller gamecontrollerdb...,https://api.github.com/repos/libsdl-org/SDL
2,4657,__label__enhancement,feature: sdl_openurl() # bug report migrated o...,https://api.github.com/repos/libsdl-org/SDL
3,4730,__label__enhancement,sdl mutex implementation subpar # bug report m...,https://api.github.com/repos/libsdl-org/SDL
4,4750,__label__enhancement,allow desktop opengl contexts egl # bug report...,https://api.github.com/repos/libsdl-org/SDL


In [82]:
df_grp = df_all.groupby(['repo_name']).agg({'descriptions':'count'}).reset_index()
# df_grp.columns = ['Labels','Row Count']
# print(df.head())
df_grp

,repo_name,descriptions
0,https://api.github.com/repos/Kaiserreich/Kaise...,7476
1,https://api.github.com/repos/Microsoft/vscode,8165
2,https://api.github.com/repos/Regalis11/Barotrauma,3769
3,https://api.github.com/repos/dbeaver/dbeaver,6099
4,https://api.github.com/repos/godotengine/godot,13535
5,https://api.github.com/repos/hashicorp/terraform,6816
6,https://api.github.com/repos/libsdl-org/SDL,3379
7,https://api.github.com/repos/microsoft/vscode,8288
8,https://api.github.com/repos/qgis/QGIS,4609
9,https://api.github.com/repos/rstudio/rstudio,2753


In [83]:
repo_names = list(df_all['repo_name'].unique())
repo_names_clean = [repo_name.split('/')[-1] for repo_name in repo_names]
repo_names_clean

['SDL',
 'dbeaver',
 'rstudio',
 'symfony',
 'godot',
 'terraform',
 'QGIS',
 'vscode',
 'vscode',
 'tgstation',
 'Barotrauma',
 'Kaiserreich-4']

In [84]:
path_test = r"..\temp\temp_eval.txt"
path_train = r"..\temp\temp_train.txt"
data_out_path = r"..\results\REPO_OUTPUT_aggregated.json"
f_out = data_out_path
output = []

for i in range(len(repo_names)):
    repo_name_clean= repo_names_clean[i]
    repo_name = repo_names[i]

    repo_df = df_all[df_all['repo_name']==repo_name]
    # split dataset to train and dev samples
    RANDOM_SEED = 12
    x_train, x_test, y_train, y_test = train_test_split(repo_df['descriptions'], repo_df['labels'],
                                                 test_size=0.5,
                                                 random_state=RANDOM_SEED)
    df_train_r = pd.concat([x_train, y_train], axis=1).reset_index(drop=True)
    df_eval_r = pd.concat([x_test, y_test], axis=1).reset_index(drop=True)

    df_train_r['data'] = df_train_r['labels'] + " " + df_train_r['descriptions']
    df_train_r = df_train_r[['data']]

    df_eval_r['data'] = df_eval_r['labels'] + " " + df_eval_r['descriptions']
    df_eval_r = df_eval_r[['data']]



    print("Creating test file")
    df_eval_r.to_csv(path_test,index=False,header=False,encoding='utf-8-sig')

    # now perform an eval with the eval dataset,
    
    # then combine the train repo with the overall training dataset 
    # now evaluate the performance of the larger trained dataset

    print("start testing...")
    print(f"test {repo_name_clean}")
    res = model.test(path_test)
    print(res)
    # get benchmarks
    precision = res[1]
    recall = res[2]
    f1 = 2 * ((precision * recall) / (precision + recall))
    result_balanced = {
        'F1': f1,
        'Recall': recall,
        'Precision': precision
    }

    print("testing over")
    # print(json.dumps(result_balanced, indent=4))

    df_train_repo_all = df_train.append(df_train_r)
    df_train_repo_all.to_csv(path_train,index=False,header=False,encoding='utf-8-sig')

    model_2 = fasttext.train_supervised(input=path_train, lr=1.0, epoch=25, wordNgrams=2)
    res = model_2.test(path_test)
    print(f"augment {repo_name_clean}")
    print(res)

    precision = res[1]
    recall = res[2]
    f1 = 2 * ((precision * recall) / (precision + recall))

    result_augmented = {
        'F1': f1,
        'Recall': recall,
        'Precision': precision
    }
    print("Augment over")
    # print(json.dumps(result_augmented, indent=4))

    output_temp = {
        'RESULTS FOR': repo_name_clean,
        'result_balanced': result_balanced,
        'result_augmented': result_augmented
    }

    output.append(output_temp)

dump = json.dumps(output, indent=4)
print(dump)
# write to output
print("Writing output to file")
# f_out = data_out_path + f"\REPO_output.txt"
o = open(f_out, 'w', encoding="UTF-8")
o.write(dump)
o.close()
# except:
# print("An Error occurred")
# # in any case delete existing temporary files
# finally:
print("Deleting tmp files")
if os.path.exists(path_train):
    os.remove(path_train)
if os.path.exists(path_test):
    os.remove(path_test)
print("Exit.")

    

Creating test file
start testing...
test SDL
(1, 1.0, 1.0)
testing over
augment SDL
(1, 1.0, 1.0)
Augment over
Creating test file
start testing...
test dbeaver
(415, 0.4506024096385542, 0.4506024096385542)
testing over
augment dbeaver
(415, 0.6457831325301204, 0.6457831325301204)
Augment over
Creating test file
start testing...
test rstudio
(79, 0.5949367088607594, 0.5949367088607594)
testing over
augment rstudio
(79, 0.7215189873417721, 0.7215189873417721)
Augment over
Creating test file
start testing...
test symfony
(148, 0.5135135135135135, 0.5135135135135135)
testing over
augment symfony
(148, 0.8986486486486487, 0.8986486486486487)
Augment over
Creating test file
start testing...
test godot
(633, 0.655608214849921, 0.655608214849921)
testing over
augment godot
(633, 0.8609794628751974, 0.8609794628751974)
Augment over
Creating test file
start testing...
test terraform
(164, 0.7012195121951219, 0.7012195121951219)
testing over
augment terraform
(164, 0.7804878048780488, 0.780487804

In [41]:
labels_list = list(df_all['labels'].unique())
labels_list

['__label__enhancement', '__label__bug', '__label__question']

# Getting the Confusion Matrix

In [42]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

In [44]:
list(recall)

[nan, 0.7156233279828785, nan]

In [62]:
path_test = r"..\temp\temp_eval.txt"
path_train = r"..\temp\temp_train.txt"
data_out_path = r"..\results\repo_output_confusionMATRIX.json"
f_out = data_out_path
output = []

for i in range(len(repo_names)):
    repo_name_clean= repo_names_clean[i]
    repo_name = repo_names[i]

    repo_df = df_all[df_all['repo_name']==repo_name]
    # split dataset to train and dev samples
    RANDOM_SEED = 12
    x_train, x_test, y_train, y_test = train_test_split(repo_df['descriptions'], repo_df['labels'],
                                                 test_size=0.5,
                                                 random_state=RANDOM_SEED)
    df_train_r = pd.concat([x_train, y_train], axis=1).reset_index(drop=True)
    df_eval_r = pd.concat([x_test, y_test], axis=1).reset_index(drop=True)

    df_train_r['data'] = df_train_r['labels'] + " " + df_train_r['descriptions']
    df_train_r = df_train_r[['data']]

    df_eval_r['data'] = df_eval_r['labels'] + " " + df_eval_r['descriptions']
    # df_eval_r = df_eval_r[['data']]



    print("Creating test file")
    df_eval_r.to_csv(path_test,index=False,header=False,encoding='utf-8-sig')

    # now perform an eval with the eval dataset,
    
    # then combine the train repo with the overall training dataset 
    # now evaluate the performance of the larger trained dataset

    print("start testing...")
    print(f"test {repo_name_clean}")

    predicted_labels = []
    actual_labels =list(df_eval_r['labels'])
    for y in array(df_eval_r['descriptions']):
        predicted_labels.append(model.predict(y))
    predicted_labels = [label[0] for label in predicted_labels]
    cm = confusion_matrix(y_true = actual_labels,y_pred=predicted_labels,labels=labels_list)
    
    TP = np.diag(cm)
    FP = np.sum(cm, axis=0) - TP
    FN = np.sum(cm, axis=1) - TP

    precision = TP/(TP+FP)
    recall = TP/(TP+FN)

    f1 = 2 * ((precision * recall) / (precision + recall))
    result_balanced = {
        'labels_list': labels_list,
        'F1': list(f1),
        'Recall': list(recall),
        'Precision': list(precision)
    }

    print("testing over")
    # print(json.dumps(result_balanced, indent=4))

    df_train_repo_all = df_train.append(df_train_r)
    df_train_repo_all.to_csv(path_train,index=False,header=False,encoding='utf-8-sig')

    model_2 = fasttext.train_supervised(input=path_train, lr=1.0, epoch=25, wordNgrams=2)
    # res = model_2.test(path_test)


    predicted_labels = []
    actual_labels =list(df_eval_r['labels'])
    for y in array(df_eval_r['descriptions']):
        predicted_labels.append(model.predict(y))
    predicted_labels = [label[0] for label in predicted_labels]
    cm = confusion_matrix(y_true = actual_labels,y_pred=predicted_labels,labels=labels_list)
    
    TP = np.diag(cm)
    FP = np.sum(cm, axis=0) - TP
    FN = np.sum(cm, axis=1) - TP

    precision = TP/(TP+FP)
    recall = TP/(TP+FN)

    f1 = 2 * ((precision * recall) / (precision + recall))
    result_augmented = {
        'labels_list': labels_list,
        'F1': list(f1),
        'Recall': list(recall),
        'Precision': list(precision)
    }


    print(f"augment {repo_name_clean}")
    print("Augment over")
    # print(json.dumps(result_augmented, indent=4))

    output_temp = {
        'RESULTS FOR': repo_name_clean,
        'result_balanced': result_balanced,
        'result_augmented': result_augmented
    }

    output.append(output_temp)

dump = json.dumps(output, indent=4)
print(dump)
# write to output
print("Writing output to file")
# f_out = data_out_path + f"\REPO_output.txt"
o = open(f_out, 'w', encoding="UTF-8")
o.write(dump)
o.close()
# except:
# print("An Error occurred")
# # in any case delete existing temporary files
# finally:
print("Deleting tmp files")
if os.path.exists(path_train):
    os.remove(path_train)
if os.path.exists(path_test):
    os.remove(path_test)
print("Exit.")


Creating test file
start testing...
test SDL


C:\Users\Ben\AppData\Local\Temp/ipykernel_17748/3591318134.py:51: RuntimeWarning: invalid value encountered in true_divide
  recall = TP/(TP+FN)


testing over


C:\Users\Ben\AppData\Local\Temp/ipykernel_17748/3591318134.py:83: RuntimeWarning: invalid value encountered in true_divide
  recall = TP/(TP+FN)


augment SDL
Augment over
Creating test file
start testing...
test dbeaver
testing over
augment dbeaver
Augment over
Creating test file
start testing...
test rstudio
testing over


...

C:\Users\Ben\AppData\Local\Temp/ipykernel_17748/3591318134.py:83: RuntimeWarning: invalid value encountered in true_divide
  recall = TP/(TP+FN)


augment Barotrauma
Augment over
Creating test file
start testing...
test Kaiserreich-4


C:\Users\Ben\AppData\Local\Temp/ipykernel_17748/3591318134.py:51: RuntimeWarning: invalid value encountered in true_divide
  recall = TP/(TP+FN)


testing over
augment Kaiserreich-4
Augment over
[
    {
        "RESULTS FOR": "SDL",
        "result_balanced": {
            "labels_list": [
                "__label__enhancement",
                "__label__bug",
                "__label__question"
            ],
            "F1": [
                0.20952380952380953,
                0.9083085071168487,
                NaN
            ],
            "Recall": [
                0.25287356321839083,
                0.8558951965065502,
                NaN
            ],
            "Precision": [
                0.17886178861788618,
                0.9675599435825106,
                0.0
            ]
        },
        "result_augmented": {
            "labels_list": [
                "__label__enhancement",
                "__label__bug",
                "__label__question"
            ],
            "F1": [
                0.20952380952380953,
                0.9083085071168487,
                NaN
            ],
            "Recal

C:\Users\Ben\AppData\Local\Temp/ipykernel_17748/3591318134.py:83: RuntimeWarning: invalid value encountered in true_divide
  recall = TP/(TP+FN)


In [85]:
output

[{'RESULTS FOR': 'SDL',
  'result_balanced': {'F1': 1.0, 'Recall': 1.0, 'Precision': 1.0},
  'result_augmented': {'F1': 1.0, 'Recall': 1.0, 'Precision': 1.0}},
 {'RESULTS FOR': 'dbeaver',
  'result_balanced': {'F1': 0.4506024096385542,
   'Recall': 0.4506024096385542,
   'Precision': 0.4506024096385542},
  'result_augmented': {'F1': 0.6457831325301204,
   'Recall': 0.6457831325301204,
   'Precision': 0.6457831325301204}},
 {'RESULTS FOR': 'rstudio',
  'result_balanced': {'F1': 0.5949367088607594,
   'Recall': 0.5949367088607594,
   'Precision': 0.5949367088607594},
  'result_augmented': {'F1': 0.7215189873417721,
   'Recall': 0.7215189873417721,
   'Precision': 0.7215189873417721}},
 {'RESULTS FOR': 'symfony',
  'result_balanced': {'F1': 0.5135135135135135,
   'Recall': 0.5135135135135135,
   'Precision': 0.5135135135135135},
  'result_augmented': {'F1': 0.8986486486486488,
   'Recall': 0.8986486486486487,
   'Precision': 0.8986486486486487}},
 {'RESULTS FOR': 'godot',
  'result_balanc

In [86]:
names = []
lables = []
f1 = []
recall = []
precision = []
catagory = []

for results in output:
    # repo names
    results_for = results['RESULTS FOR']
    temp= list(results_for) * 6
    names+=names

    temp = ['result_balanced']*3 +['result_augmented'] *3
    catagory+=temp

    
    f1+=results['result_balanced']['F1']
    f1+=results['result_augmented']['F1']

    recall+=results['result_balanced']['Recall']
    recall+=results['result_augmented']['Recall']

    precision+=results['result_balanced']['Precision']
    precision+=results['result_augmented']['Precision']



TypeError: 'float' object is not iterable

In [ ]:
my_df = pd.DataFrame([names,catagory,f1,recall,precision]).T

In [ ]:
my_df

,0,1,2,3,4
0,None,result_balanced,0.209524,0.252874,0.178862
1,None,result_balanced,0.908309,0.855895,0.96756
2,None,result_balanced,NaN,NaN,0.0
3,None,result_augmented,0.209524,0.252874,0.178862
4,None,result_augmented,0.908309,0.855895,0.96756
...,...,...,...,...,...
67,None,result_balanced,0.834243,0.715623,1.0
68,None,result_balanced,NaN,NaN,0.0
69,None,result_augmented,NaN,NaN,0.0
70,None,result_augmented,0.834243,0.715623,1.0
